# COGS 108 - Data Checkpoint

# Names

- Jamie Wei
- Alexis Garduno
- James Daza
- Aleksander Archipov

<a id='research_question'></a>
# Research Question

- Does the number of times a player is traded predict that player's performance (e.g. average total points scored/game, average minutes played/game, etc) in an NBA game? Also, does the turn-over rate (e.g. number of players traded within a team/season) of a NBA team affect the likelihood that the team will reach the NBA finals (evidenced by the last 20 years of NBA games)?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: nba_api (replace with the dataset we generated from nba_api) 
- Link to the dataset: https://github.com/swar/nba_api
- Number of observations: N/A
    - API Client that allows access to various NBA's stats API endpoints

- Dataset Name: NBA Stats (replace with boxscore and team performance datasets)
- Link to the dataset: https://www.nba.com/stats/
- Number of observations: No, this should not be NA. Please update based on obtained dataset. 
    - This question is referring to the dataset we have pulled out from the NBA API


1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [ ]:
import pandas as pd
import numpy as np
import time

In [18]:
#NBA team players
from nba_api.stats.static import players

from nba_api.stats.endpoints import playercareerstats
# Anthony Davis
nba_players = players.get_players()
career = playercareerstats.PlayerCareerStats(player_id='1630188')

# Loop through each dictionary and grab the id


AttributeError: 'list' object has no attribute 'get_dataframe'

In [19]:
nba_players.get_data_frames()[0]

AttributeError: 'list' object has no attribute 'get_data_frames'

In [3]:
#Import the roster of teams from the NBA API
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

In [4]:
#Obtain a full list of all abbreviations - will need abbreviations to identify team statistics
nba_teams_df=pd.DataFrame(nba_teams)
team_id=nba_teams_df['id'] #this is the unique team id
team_id_random=np.random.choice(team_id,10,replace=False) #identify ten team ids
nba_teams_rdf=nba_teams_df[nba_teams_df['id'].isin(team_id_random)] #df of 10 randomly selected teams
nba_teams_rdf

,id,full_name,abbreviation,nickname,city,state,year_founded
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
11,1610612748,Miami Heat,MIA,Heat,Miami,Florida,1988
14,1610612751,Brooklyn Nets,BKN,Nets,Brooklyn,New York,1976
20,1610612757,Portland Trail Blazers,POR,Trail Blazers,Portland,Oregon,1970
21,1610612758,Sacramento Kings,SAC,Kings,Sacramento,California,1948
23,1610612760,Oklahoma City Thunder,OKC,Thunder,Oklahoma City,Oklahoma,1967
24,1610612761,Toronto Raptors,TOR,Raptors,Toronto,Ontario,1995
25,1610612762,Utah Jazz,UTA,Jazz,Utah,Utah,1974


In [5]:
#Pull all games for all ten teams

#Documentation for this endpoint: 
#https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md
from nba_api.stats.endpoints import leaguegamefinder

# Query for games from the League Game Finder
gamefinder=pd.DataFrame()
for i in team_id_random:
    time.sleep(1) #delay to prevent being blocked from the API
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=[i]).get_data_frames()[0] #parameter of team ids given
    gamefinder = pd.concat([df,gamefinder])

In [17]:
#Game Statistics

#One row corresponds to one game and one team.
#There will be two rows per game, since there are two teams that played each other.
#Will need to exclude duplicate rows (XXXX will remove duplicate rows)
print(list(set(gamefinder.TEAM_ID))) #confirmed that identified 10 different teams
print(gamefinder.shape) #31,386 games

##Game Finder Dataset: This dataset will be used as the outcome when we look at the association between 
##the exposure and outcome relationship. 
gamefinder

[1610612739, 1610612741, 1610612745, 1610612748, 1610612751, 1610612757, 1610612758, 1610612760, 1610612761, 1610612762]
(33552, 28)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612760,OKC,Oklahoma City Thunder,0022100827,2022-02-09,OKC vs. TOR,L,240,98,...,0.857,14.0,31.0,45,28,11,4,17,17,-19.0
1,22021,1610612760,OKC,Oklahoma City Thunder,0022100813,2022-02-07,OKC vs. GSW,L,240,98,...,0.765,18.0,32.0,50,21,8,2,17,14,-12.0
2,22021,1610612760,OKC,Oklahoma City Thunder,0022100801,2022-02-05,OKC @ SAC,L,240,103,...,0.625,7.0,34.0,41,23,8,1,14,16,-10.0
3,22021,1610612760,OKC,Oklahoma City Thunder,0022100795,2022-02-04,OKC @ POR,W,240,96,...,0.714,13.0,39.0,52,24,5,6,14,18,3.0
4,22021,1610612760,OKC,Oklahoma City Thunder,0022100778,2022-02-02,OKC @ DAL,W,266,120,...,0.840,14.0,36.0,50,25,6,8,18,16,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3414,21983,1610612751,NJN,New Jersey Nets,0028300050,1983-11-05,NJN vs. PHL,L,240,112,...,0.630,21.0,24.0,45,30,9,19,18,28,NaN
3415,21983,1610612751,NJN,New Jersey Nets,0028300039,1983-11-03,NJN vs. CHI,W,240,112,...,0.757,17.0,40.0,57,29,12,10,19,26,NaN
3416,21983,1610612751,NJN,New Jersey Nets,0028300030,1983-11-01,NJN vs. CLE,W,240,125,...,0.767,14.0,25.0,39,26,8,3,23,27,NaN
3417,21983,1610612751,NJN,New Jersey Nets,0028300012,1983-10-29,NJN @ CHI,L,240,97,...,0.610,8.0,28.0,36,20,7,5,22,33,NaN


In [11]:
#get game ids for the last five years

#for now, let's focus on the last five seasons for ease
from nba_api.stats.endpoints import playergamelogs

#generate a parameter dataframe to define timeframe
#Is this timeframe correct? What is the timeframe that the season normally runs from?
season_parameter_df=pd.DataFrame({'Season':['2016-17','2017-18','2018-19','2019-20','2020-21'], 
                    'Date_From':['9/01/2016','9/01/2017','9/01/2018','9/01/2019','9/01/2020'],
                    'Date_To':['8/31/2017','8/31/2018','8/31/2019','8/31/2020','8/31/2021']})

#will obtain no game ids, without the season_nullable and date_nullable items selected
logsdf=pd.DataFrame()
for i in list(range(1, 5)):
    time.sleep(1)
    season=season_parameter_df.iloc[i][0]
    date_from=season_parameter_df.iloc[i][1]
    date_to=season_parameter_df.iloc[i][2]
    
    logs = pd.DataFrame(playergamelogs.PlayerGameLogs(
        season_nullable = season,
        date_from_nullable = date_from,                                                     
        date_to_nullable = date_to
    ).player_game_logs.get_data_frame())
    logsdf = pd.concat([logs,logsdf])

In [ ]:
#unique set of game ids
game_ids = list(set(logsdf['GAME_ID'])) #not sure if we need game_ids
player_ids = pd.DataFrame(list(set(logsdf['PLAYER_ID'])))
print(pd.DataFrame(game_ids).shape) #4599 game ids
print(player_ids.shape) #875 unique players
print(logsdf.shape) #97,655 rows where each row is for each player in each game. 
games_ids

In [9]:
#preview, likely not use these columns
logsdf.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
       'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

In [84]:
#Copy Logs DataFrame and remove TEAM ID's that were not selected
logsRandTeamDf = logsdf.copy()
logsRandTeamDf = logsRandTeamDf[logsRandTeamDf['TEAM_ID'].isin(team_id_random)]

In [86]:
# Restrict to the player stats we want
logsRandTeamDf = logsRandTeamDf[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']]
logsRandTeamDf

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,MIN,FG_PCT,FT_PCT,PTS,AST,REB,STL,BLK,PLUS_MINUS
0,2020-21,1629013,Landry Shamet,25.116667,0.500,0.000,12,3,2,0,0,11
2,2020-21,1629650,Moses Brown,37.850000,0.632,0.000,24,3,18,0,7,0
3,2020-21,1626181,Norman Powell,33.050000,0.556,1.000,19,1,3,0,0,14
7,2020-21,201145,Jeff Green,22.400000,0.667,1.000,13,0,2,1,1,-12
10,2020-21,1629216,Gabe Vincent,36.733333,0.500,0.000,15,10,6,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
26097,2017-18,201935,James Harden,36.400000,0.435,0.750,27,11,6,1,0,1
26098,2017-18,201601,Luc Mbah a Moute,23.383333,0.667,0.000,14,0,4,0,0,4
26101,2017-18,2747,JR Smith,21.916667,0.571,1.000,10,1,4,0,0,7
26103,2017-18,2544,LeBron James,41.200000,0.632,1.000,29,9,16,0,2,3


In [54]:
#import boxscore, player statistics within each game
#https://en.wikipedia.org/wiki/Box_score_(baseball)#:~:text=A%20box%20score%20is%20a,the%20box%20score%20in%201858.
from nba_api.stats.endpoints import BoxScoreAdvancedV2

# Takes a LONG time to run
boxscfinder=pd.DataFrame()
for i in game_ids:
        time.sleep(1) #delay to prevent being blocked from the API
        boxscore = BoxScoreAdvancedV2(game_id=i).player_stats.get_data_frame()
        boxscfinder = pd.concat([boxscore,boxscfinder])

KeyboardInterrupt: 

In [ ]:
# NEED TO RUN TOP CELL FIRST
#boxscfinder = boxscfinder[boxscfinder['TEAM_ID'].isin(team_id_random)]
#boxscfinder = boxscfinder[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']

In [ ]:
####NEXT STEPS:

####These tasks need to be completed and we can split them up.
#Task (Insert your name)

#TASK 1 (NAME): Get a final dataset for the Boxscore (verify that this runs)
#TASK 2 (NAME): Only keep the boxscore entries that match the teams identified
#in the team_id_random ARRAY; generate a file

#TASK 3 (Jamie): Run this notebook and restrict the dataset
#named "boxscfinder" to the parameters we need based on what you 
#identified. Note, Jamie, when you restrict the parameter, it would
#be helpful in the code to name the variables. 

#TASK 4 (NAME): Restrict the "gamefinder" dataframe to the "team performance" metrics

#TASK 5 (Alexis): Look at the final boxscore dataset and define the traded variable

#TASK 6 (Alek/other): Estimate the data distributions of the 
#the parameters that we are examining. 
#Depends on Task 3 and 4 being complete. 
#Generate descriptive statistics for the parameters that we list.


# Data Cleaning

(Alek and others) Describe. In class, data cleaning examples were to handle missing values, recode variables (generate new variables), and plot distribution.

In [87]:
#Saving the Data to a csv file

logsRandTeamDf.to_csv("logPlayerStats.csv")

# Run later
# boxscfinder.to_csv("boxscfinder.csv")